In [3]:
import os
import skvideo.io
import tqdm
import sys
import numpy as np
import csv
import cv2
import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
LENGTH=10 #in secs
FRAME_RATE=24 # Frame rate of the frame videos
SAMPLE_RATE=2 # Sampling rate for feature extraction
NUM_FRAMES=int(LENGTH*FRAME_RATE/SAMPLE_RATE)

In [4]:
# USER INPUT- data_dir and features_dir
# data_dir- path to frame videos. Required only for feature extraction
# features_dir- path to save or load vgg face features
curr=os.getcwd()
features_path=repo_path+'/features' #  Folder containing all features audio and video

In [5]:

repo_path=curr.split('/code')[0]
data_dir=repo_path+'/DIFv2'+'/'+str(LENGTH)+'/frame_video' # USER INPUT Path to frame videos directory. Required only for feature extraction
features_dir=features_path+'/'+str(LENGTH)+'/vgg_face7' # USER INPUT Path to save/load vgg face features

# Extracting and saving VGG Face features, layer-fc7
* Frame video- 10 secs, 24 frame rate- Total 240 frames
* Features are extracted for sampled frames in a frame video.
* Frame feature extraction using VGG Model:- https://gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9
    - Input dim 3 * 224 * 224
    - Output feature length 4096
* A sampling rate of 2 will generate the feature vector of dimension [120,4096].


 Skip this section if features are already extracted.

In [ ]:
#USER INPUT:- Path to vgg face model
model_path=repo_path+'/saved_models/vgg_face/vgg-face-keras.h5'

In [6]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
import keras
from keras.models import load_model



In [7]:
from keras.optimizers import Adam
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout,Activation
from keras import backend as K
K.set_image_data_format('channels_first')

def vgg_face(weights_path=None):
    img = Input(shape=(3, 224, 224))

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Conv2D(64, (3, 3), activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Conv2D(64, (3, 3), activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Conv2D(128,(3, 3), activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Conv2D(128, (3, 3), activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Conv2D(256, (3, 3), activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Conv2D(256, (3, 3), activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Conv2D(256, (3, 3), activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Conv2D(512, (3, 3), activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Conv2D(512, (3, 3), activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Conv2D(512, (3, 3), activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Conv2D(512, (3, 3), activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Conv2D(512, (3, 3), activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Conv2D(512, (3, 3), activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

    fc6 = Conv2D(4096, (7, 7), activation='relu', name='fc6')(pool5)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Conv2D(4096, (1, 1), activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    fc8 = Conv2D(2622, (1, 1), name='fc8')(fc7_drop)
    flat = Flatten()(fc8)
    out = Activation('softmax')(flat)

    model = Model(inputs=img, outputs=out)

    if weights_path:
        model.load_weights(weights_path)

    return model

In [8]:
base_model=vgg_face(weights_path=model_path)
base_model.summary()
layer_name = 'fc7'
model = Model(inputs=base_model.input,outputs=base_model.get_layer(layer_name).output)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0   

In [9]:
h=224
w=224
output_len=4096

#input video- list of numpy arrays [224,224,3] * num frames 
def get_features(video):
    feature=[]
    for i in range(len(video)):
        feature.append(model.predict(np.expand_dims(video[i], axis=0)).reshape(output_len))
        
    return feature

In [10]:
def video_reader(path): 
    """
        path: path to the video file
        return numpy array
    """
    arr=[]
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
  
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
  
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read()
        if success and count%SAMPLE_RATE==0:
            img=cv2.resize(image,(h,w))
            #print(img.shape)
            channel_ch=np.rollaxis(img, 2, 0)
            #print(channel_ch.shape)
            arr.append(channel_ch)
        count += 1
    return arr

In [12]:
video_list=os.listdir(data_dir)
for vid in tqdm.tqdm(video_list):
    if vid[-4:]=='.mp4':
        filename=vid[:-4]
        if not os.path.isfile(features_dir+'/'+filename+'.npy'):
            video_array=video_reader(data_dir+'/'+vid)
            if len(video_array)!=NUM_FRAMES:
                print("number frames not match , frames = ",len(video_array),", video is ",vid)
                continue 
            
            feature=np.array(get_features(video_array))
            
        #saving features
    
            np.save(os.path.join(features_dir, filename),feature)
    
    


 28%|██▊       | 1814/6427 [27:49<7:29:37,  5.85s/it]


 33%|███▎      | 2100/6427 [55:44<7:02:01,  5.85s/it]


 37%|███▋      | 2378/6427 [1:22:51<6:39:51,  5.93s/it]


 41%|████▏     | 2654/6427 [1:49:39<6:06:51,  5.83s/it]


 46%|████▌     | 2930/6427 [2:16:27<5:39:16,  5.82s/it]


...

 84%|████████▍ | 5414/6427 [6:17:42<1:38:24,  5.83s/it]


 89%|████████▊ | 5690/6427 [6:44:29<1:11:34,  5.83s/it]


 93%|█████████▎| 5972/6427 [7:11:53<44:16,  5.84s/it]


 97%|█████████▋| 6258/6427 [7:40:04<16:50,  5.98s/it]


100%|██████████| 6427/6427 [7:57:25<00:00,  4.46s/it]


# Data generator

Link-https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [4]:
'''
Input-  csv_file
Output- partition train, val test. Each partition consists of list of .npy files and dictionary of labels.
'''
def train_test_split(csv_path):
    label={'Drunk':1, 'Sober':0}
    partition={}
    train={}
    val={}
    test={}
    
    train_list=[]
    val_list=[]
    test_list=[]
    train_label={}
    val_label={}
    test_label={}
    
    with open(csv_path) as csvfile:
        reader=csv.reader(csvfile,delimiter=',')
        for row in reader:
            filename=row[2]
            filename=filename[:-4]
            if row[0]=='train':
                train_label[filename]=label[row[1]]
                train_list.append(filename)
            elif row[0]=='val':
                val_label[filename]=label[row[1]]
                val_list.append(filename)
            elif row[0]=='test':
                test_label[filename]=label[row[1]]
                test_list.append(filename)
            else:
                print("Error in label")
                return None
    train['list']=train_list
    val['list']=val_list
    test['list']=test_list
    
    train['label']=train_label
    val['label']=val_label
    test['label']=test_label
    
    partition['train']=train
    partition['val']=val
    partition['test']=test
    
    return partition
def count_classes(d):
    values=list(d.values())
    zeros=values.count(0)
    return (zeros,len(values)-zeros)
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels,datapath, batch_size=32, dim=(24,1000),n_classes=2, shuffle=True):
        'Initialization'        
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        #self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path=datapath

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        import gc
        gc.collect()
        return X, y
    
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load(self.path+'/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

# 3 Training


## 3.1 Model creation and summary
Batch normaliztion
https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c

In [5]:
from keras.optimizers import Adam
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from time import time
import gc

In [6]:
def create_model(num_class,lstm_units,dropout,input_shape=(None,1000)):
    X=Input(shape=input_shape)
    norm=BatchNormalization()(X)
    feat=LSTM(units=lstm_units)(norm)
    drop=Dropout(rate=dropout)(feat)
    prob=Dense(num_class, activation='sigmoid')(drop)
    return Model(inputs = X, outputs = prob)
    
    

In [7]:
lstm_units=128
dropout=.2
hp=3
model=create_model(2,lstm_units,dropout,(NUM_FRAMES,4096))
model_path=repo_path+'/saved_models/vgg_lstm/'+str(LENGTH)+'/hp'+str(hp)# USER INPUT, path to save/load model
model.summary()
print("Must check the csv path...........")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120, 4096)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 120, 4096)         16384     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               2163200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 2,179,842
Trainable params: 2,171,650
Non-trainable params: 8,192
_________________________________________________________________
Must check the csv path...........


## 3.2 Fit the model

Class Imbalance 
https://groups.google.com/forum/#!topic/keras-users/MUO6v3kRHUw

class_weight in keras
https://keras.io/models/model/

tensorboard 
https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/

save every k-epochs

In [ ]:
#USER INPUT
split_path = repo_path+'/DIFv2/'+str(LENGTH)+'/train_test_sets/1/split_4540_642_948.csv'# or enter path to the split.csv in the parent directory 

In [11]:
def load_keras_model(path):
    if os.path.isfile(path):
        return load_model(path)
#Loading data filenames split

partition=train_test_split()
print("Number of training examples ")
print(len(partition['train']['list']))
print("Number of validation examples ")
print(len(partition['val']['list']))

params = {'datapath':features_dir ,
          'dim': (NUM_FRAMES,4096),
          'batch_size': 64,
          'n_classes': 2,
          'shuffle': False}
    
#weights for imbalance classes
count=count_classes(partition['train']['label'])
print("Class instances in training class.\n Sober:",count[0]," Drunk:",count[1])
weight_0=float(count[0]+count[1])/float(count[0])
weight_1=float(count[0]+count[1])/float(count[1])
class_weight={0:weight_0, 1:weight_1}

#instances in val set
count=count_classes(partition['val']['label'])
print("Class instances in val class.\n Sober:",count[0]," Drunk:",count[1])

#instances in test set
count=count_classes(partition['test']['label'])
print("Class instances in test class.\n Sober:",count[0]," Drunk:",count[1])


Number of training examples 
4540
Number of validation examples 
642
Class instances in training class.
 Sober: 1045  Drunk: 3495
Class instances in val class.
 Sober: 321  Drunk: 321
Class instances in test class.
 Sober: 306  Drunk: 642


In [ ]:

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=["accuracy"])

#saving best model
checkpoint = ModelCheckpoint(model_path+'/model-{epoch:03d}-{val_acc:03f}.h5', verbose=1, monitor='val_acc',save_best_only=False, mode='max',period=10)


#tensorboard
tensorboard = TensorBoard(log_dir=model_path+"/log/{}".format(time()))

train_generator=DataGenerator(partition['train']['list'],partition['train']['label'], **params)
val_generator=DataGenerator(partition['val']['list'],partition['val']['label'], **params)
print("generator created")
model.fit_generator(generator=train_generator,epochs=180,validation_data=val_generator,
                    use_multiprocessing=True,
                    workers=4,callbacks=[checkpoint,tensorboard],class_weight=class_weight)